# 학습데이터 수집

- id(정수로 0부터 가능)로 얼굴인식 데이터 수집

In [ ]:
import cv2
import os

cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# For each person, enter one numeric face id 
face_id = input('\n enter user id end press <return> ==>  ')
print("\n [INFO] Initializing face capture. Look the camera and wait ...")

# Initialize individual sampling face count
count = 0
while(True):
    ret, img = cam.read()
    #img = cv2.flip(img, -1) # flip video image vertically
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])
        cv2.imshow('image', img)
    k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 500: # Take 30 face sample and stop video
        break
# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()

# 수집된 데이터 학습

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os

# Path for face image database
path = 'dataset'
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");

# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return faceSamples,ids

print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml') # recognizer.save() worked on Mac, but not on Pi
# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))

# openCV를 이용하여 실시간 얼굴인식

- id를 부여(0~3)하여 조원인식
- 키보드 i 버튼을 이용하여 smile, angry 필터효과 on/off
- smile, angry 각 다른 필터적용

In [1]:
import cv2, os
import dlib
import sys
import numpy as np
import time
import datetime
import os
from keras.utils import np_utils
import tensorflow as tf
import keras
import imutils
import pytesseract
from keras.models import load_model
from twilio.rest import Client

# 마스크 라벨
labels_dict={1:'smile', 0:'angry'}
color_dict={0:(0,255,0),1:(0,0,255)}

# avi
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

# 텍스트표시
font = cv2.FONT_ITALIC

# face 분류기 로드
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml') 

# 실시간 개인분류 학습모델
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')

# open CV
smile_detact = False
source=cv2.VideoCapture(0)
time.sleep(3) #warming up
source.set(3,640)
source.set(4,480)
record = False

# Define min window size to be recognized as a face
minW = 0.1*source.get(3)
minH = 0.1*source.get(4)

# 스마일 - 모델
model = load_model('059-0.1275.model')

# 개인구분 ID 및 이름
id = 0, 1, 2, 3
# names related to ids: example ==> loze: id=1,  etc
names = ['SY', 'JM', 'HS', 'BJ']

# twilio
#account_sid = 'Twilio API'
#auth_token = 'Twilio API'

# mask_이미지사진
s_mask = cv2.imread('./mask/7.png') # smile 필터
a_mask = cv2.imread('./mask/10.png') # angry 필터
sh_mask, sw_mask = s_mask.shape[:2]
ah_mask, aw_mask = s_mask.shape[:2]

if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file')

while True:
    ret,frame = source.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_rects = face_cascade.detectMultiScale(gray,
                                               scaleFactor=1.2,
                                               minNeighbors=5,
                                               minSize=(int(minW), int(minH)))
    if not ret:
        break
        #frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
    
    if smile_detact:
        info = "Smile Detention ON"
    else:
        info = "Smile Detection OFF"
    
    cv2.putText(frame, info, (5,15), font, 0.5, (255,0, 255),1)
    
    for (x, y, w, h) in face_rects:
        center = (x + w // 2, y + h // 2)
        #frame = cv2.ellipse(frame, center, (w // 2, h // 2), 0, 0, 360, (255, 0, 255), 4)
        #cv2.putText(frame, "Detected Face", (x-5, y-5), font, 0.5, (255,255,0),2)
        
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
        #cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
        #cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
        
        #cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        
        # Check if confidence is less them 100 ==> "0" is perfect match
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(frame, str(id), (x+90, y-45), font, 1, (0,216,255), 3)
        cv2.putText(frame, str(confidence), (x+80,y-30), font, 0.5, (0,216,255), 1)
                                               
        if(labels_dict[label] == 'angry'):
            print("웃어보세요")
            
        else:
            #cv2.imwrite("../data/capture/" + str(now) + ".png", frame)
            #client = Client(account_sid, auth_token)
            #message = client.messages.create(
            #            to="+821021277767", 
            #            from_="+19092807561",
            #            body="마스크 쓰세요")
            print("웃는모습이 최고에요 / capture or send SMS") 
            
        cv2.putText(
            frame, "{}: {:.2f}%".format(labels_dict[label], np.max(result) * 100),
            (x, y-10),
            cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),2)
        
        if smile_detact: 
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            
            smiles = smile_cascade.detectMultiScale(roi_gray,
                                                    scaleFactor=1.5,
                                                    minNeighbors=15,
                                                    minSize=(25,25))
            for (xx, yy, ww, hh) in smiles: 
                #cv2.rectangle(roi_color, (xx, yy), ((xx + ww), (yy + hh)), (0, 255, 0), 2)
                
                if(labels_dict[label] == 'smile'):
                    if h > 0 and w > 0:
                        x = int(x -w*0.1)
                        y = int(y -h*0.35)
                        w = int(1.2 * w)
                        h = int(1.2 * h)
                    
                        frame_roi = frame[y:y + h, x:x + w]
                        face_mask_small = cv2.resize(s_mask, (w, h), interpolation=cv2.INTER_AREA)
                        gray_mask = cv2.cvtColor(face_mask_small, cv2.COLOR_BGR2GRAY)
                        ret, mask = cv2.threshold(gray_mask, 240, 255, cv2.THRESH_TRUNC)
                        #cv2.THRESH_BINARY_INV / cv2.THRESH_TRUNC

                    mask_inv = cv2.bitwise_not(mask)
                    masked_face = cv2.bitwise_and(face_mask_small, face_mask_small, mask=mask)
                    masked_frame = cv2.bitwise_and(frame_roi, frame_roi, mask=mask_inv)
                    frame[y:y + h, x:x + w] = cv2.add(masked_face, masked_frame)
                    
                elif(labels_dict[label] == 'angry'):
                    if h > 0 and w > 0:
                        x = int(x -w*0.1)
                        y = int(y -h*0.35)
                        w = int(1.2 * w)
                        h = int(1.2 * h)
                    
                        frame_roi = frame[y:y + h, x:x + w]
                        face_mask_small = cv2.resize(a_mask, (w, h), interpolation=cv2.INTER_AREA)
                        gray_mask = cv2.cvtColor(face_mask_small, cv2.COLOR_BGR2GRAY)
                        ret, mask = cv2.threshold(gray_mask, 240, 255, cv2.THRESH_TRUNC)
                        #cv2.THRESH_BINARY_INV / cv2.THRESH_TRUNC

                    mask_inv = cv2.bitwise_not(mask)
                    masked_face = cv2.bitwise_and(face_mask_small, face_mask_small, mask=mask)
                    masked_frame = cv2.bitwise_and(frame_roi, frame_roi, mask=mask_inv)
                    frame[y:y + h, x:x + w] = cv2.add(masked_face, masked_frame)
    
    key = cv2.waitKey(33)
    cv2.imshow('Face Detector', frame)
    now = datetime.datetime.now().strftime("%d_%H-%M-%S") #사진저장
    
    if key == 27: # ESC
        #os.startfile("capture")
        print("종료")
        break
    elif key == ord('i'):
        smile_detact = not smile_detact
    elif key == 99: # C
        print("캡쳐")
        cv2.imwrite("capture/" + str(now) + '-' + str(count) + ".png", frame)
    elif key == 115: # S
        print("녹화 시작")
        record = True
        out = cv2.VideoWriter("capture/" + str(now) + ".avi", fourcc, 20, (640, 480))
    elif key == 122: # Z
        print("녹화 중지")

        record = False
        out.release()
        
    if record == True:
        out.write(frame)

source.release()
cv2.destroyAllWindows()

웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
웃어보세요
종료
